In [30]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
#maybe not useful:
from sklearn.feature_extraction.text import TfidfVectorizer


In [10]:
from sklearn.svm import SVC

In [12]:
from sklearn.model_selection import GridSearchCV

In [2]:
x_train = pd.read_csv("../data/x_train.csv", encoding = "ISO-8859-1")
y_train = pd.read_csv("../data/y_train.csv", encoding = "ISO-8859-1")

x_val = pd.read_csv("../data/x_val.csv", encoding = "ISO-8859-1")
y_val = pd.read_csv("../data/y_val.csv", encoding = "ISO-8859-1")

In [3]:
tfidf = TfidfVectorizer()

In [4]:
tfidf_train = tfidf.fit_transform(x_train["text"])
tfidf_val = tfidf.transform(x_val["text"])

In [29]:
tfidf_train.shape

(2554, 5120)

In [6]:
non_zero_cols = tfidf_train.nnz / float(tfidf_train.shape[0])
print("Average Number of Non-Zero Elements in Vectorized Articles: {}".format(non_zero_cols))

percent_sparse = 1 - (non_zero_cols / float(tfidf_train.shape[1]))
print('Percentage of columns containing ZERO: {}'.format(percent_sparse))

Average Number of Non-Zero Elements in Vectorized Articles: 16.69224745497259
Percentage of columns containing ZERO: 0.9967397954189506


<2554x5120 sparse matrix of type '<class 'numpy.float64'>'
	with 42632 stored elements in Compressed Sparse Row format>

### Naive Bayes

In [7]:
gnb = GaussianNB()

In [36]:
gnb = gnb.fit(tfidf_train.todense(), y_train["target"])
y_pred = gnb.predict(tfidf_val.todense())

In [37]:
accuracy_score(y_pred, y_val["target"])

0.7621283255086072

In [38]:
confusion_matrix(y_val["target"],y_pred)

array([[ 51,  48],
       [104, 436]])

### Support Vector Classification

In [13]:
params = [{'class_weight': ['balanced'], 
           'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 
           'C': [10, 5, 1, 0.8, 0.5, 0.1],
          'gamma': ['scale','auto']}]

gridsearch = GridSearchCV(SVC(), params, scoring='accuracy', 
                          cv=5, verbose=1, n_jobs=-1)

In [15]:
gridsearch.fit(tfidf_train,  y_train["target"])

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   18.7s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  1.8min finished


GridSearchCV(cv=5, estimator=SVC(), n_jobs=-1,
             param_grid=[{'C': [10, 5, 1, 0.8, 0.5, 0.1],
                          'class_weight': ['balanced'],
                          'gamma': ['scale', 'auto'],
                          'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}],
             scoring='accuracy', verbose=1)

In [17]:
gridsearch.best_score_

0.868437128275968

In [18]:
gridsearch.best_params_

{'C': 5, 'class_weight': 'balanced', 'gamma': 'scale', 'kernel': 'rbf'}

In [39]:
svc= SVC(C= 5, class_weight='balanced', gamma='scale', kernel='rbf')
svc.fit(tfidf_train,  y_train["target"])

SVC(C=5, class_weight='balanced')

In [40]:
y_pred = svc.predict(tfidf_val.todense())
f1_score(y_pred, y_val["target"])

0.9367311072056239

In [41]:
confusion_matrix(y_val["target"],y_pred)

array([[ 34,  65],
       [  7, 533]])